![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio A - Introducción a Py_pol </font>
</div>

- **Fecha**: Nov 27th 2022
        
- **Alumno**: Alex Recuenco (Alejandro Gonzalez Recuenco)


# Introducción

Este es un ejercicio sencillo para empezar a utilizar el módulo *py_pol*.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from py_pol.jones_matrix import Jones_matrix
from py_pol.jones_vector import Jones_vector

In [ ]:
def label(ax, title: str):
    ax.set_title(title)
    ax.set_xlabel(r"$\alpha$ (pol)")
    ax.set_ylabel(r"$\alpha$ (ret)")


def plt_az_ellip(mesh_x, mesh_y, az_ell, *, label=label, title=None):
    (azimuth, ellipticity) = az_ell

    fig, (az_plt, el_plt) = plt.subplots(1, 2, constrained_layout=True)
    if title:
        plt.title(title)
    az_plt.pcolor(mesh_x, mesh_y, np.mod(azimuth + np.pi / 2, np.pi))
    el_plt.pcolor(mesh_x, mesh_y, np.mod(ellipticity + np.pi / 2, np.pi))
    label(az_plt, title="azimuth")
    label(el_plt, title="ellipticity")
    plt.show()

# 1. Generador de estados

Vamos a modelar un generador de estados compuesto por los mismos elementos que vemos en clase y en el laboratorio: una fuente de luz circularmente polarizada y un polarizador y una lámina de cuarto de onda rotatorios. Utilizaremos elementos ideales por simplicidad.

El ejercicio consiste en una demostración ilustrativa de las ecuaciones que relacionan los ángulos de los elementos del PSG con el acimut y el ángulo de elipticidad del estado generado. Para ello tendréis que realizar los siguientes pasos.

1. Crear un elemento *Jones_matrix* que represente al polarizador y otro al retardador. Estos elementos deben tener todas las posibles combinaciones de ángulos de giro (entre 0º y 180º) con una resolución mínima de 1º. Para esto será de gran ayuda la función *meshgrid* de *numpy*.
2. Crear la fuente circularmente polarizada como un *Jones_vector*. Multiplicar adecuadamente los elementos para obtener el estado a la salida del PSG.
3. Representar gráficamente el aimut y el ángulo de elipticidad del estado final. Para ello serán de gran ayuda los métodos *azimuth* y *ellipticity* de la subclase *Jones_vector.parameters*.
4. Representar gráficamente el resultado de aplicar las ecuaciones dadas en clase que calculan el acimut y el ángulo de elipticidad del estado final a partir de los ángulos de los elementos del PSG.
5. Analizar las diferencias.

In [ ]:
degree_to_rad = np.pi / 180
angles = np.arange(0.0, 180.0, step=1) * degree_to_rad
(a_pol, a_retarder) = np.meshgrid(angles, angles)

laser = Jones_vector("laser").circular_light()
polarizer = Jones_matrix("polarizer").diattenuator_perfect(azimuth=a_pol)
retarder = Jones_matrix("retarder").retarder_linear(azimuth=a_retarder)

output = retarder * polarizer * laser

az, ell = output.parameters.azimuth_ellipticity()
plt_az_ellip(a_pol / degree_to_rad, a_retarder / degree_to_rad, (az, ell))
plt.close()

Normally after polarizer we would get polarizer defining the azimuth and zero ellepticity. 
Then, the retarter would change the ellepticity angle as $\chi = \theta_{Q} - \theta_P$ and $\phi = \theta_P$


They look different, but this is because numbers are wriapping... I think. The direction of change in both of them is the same

In [ ]:
(az_theory, ell_theory) = (a_retarder, a_pol - a_retarder)
plt_az_ellip(a_pol / degree_to_rad, a_retarder / degree_to_rad, (az_theory, ell_theory))

Dibujamos las diferencias. Se puede ver que el color el plano, lo cual indica que la diferencia es un artefacto de la convención de ángulos. Se podría recuperar la convención, con cuidado de evaluar cómo se hace internamente, comparándolo con como lo hemos calculado externamente.

In [ ]:
plt_az_ellip(
    a_pol / degree_to_rad,
    a_retarder / degree_to_rad,
    (az_theory - az, ell_theory - ell),
    title="difference",
)

# 2. Analizador de estados

Ahora vamos a modelar el analizador de estados, de nuevo con una lámina de cuarto de onda y un polarizador rotatorios. De nuevo, serán elementos ideales.

El ejercicio consistirá en una demostración visual de que el estado analizado es aquel que tiene mayor transmisión (1 porque emplearemos elementos ideales) y su estado ortogonal la mínima (0 en el caso de elementos ideales). Para ello debéis realizar los siguientes pasos.

1. Crear dos objetos *Jones_matrix* que representen al retardador y polarizador del PSA. Escoger sus ángulos para que el estado analizado tenga un acimut y un ángulo de elipticidad conocidos pero generados aleatoriamente (por ejemplo, empleando la función *numpy.random.randint*). 
2. Crear un objeto *Jones_vector* que va a ser el estado incidente. Este objeto tiene que contener todas las combinaciones posibles de acimut y ángulo de elipticidad (entre 0º y 180º y -45º y 45º) con una resolución mínima de 1º.
3. Multiplicar los elementos de manera que se pueda calcular el estado final.
4. Calcular y representar la transmisión de intensidad del PSA.
5. Identificar los puntos de transmisión 0 y 1. Pueden ser de utilidad las funciones *argmin*, *argmax* y *unravel_index* de *numpy*.

El ejercicio se resuelve visualmente

In [ ]:
import py_pol

az_radian = np.arange(0.0, 180.0, step=1) * degree_to_rad
el_radian = np.arange(-45, 45.0, step=1) * degree_to_rad
(az_radian, el_radian) = np.meshgrid(az_radian, el_radian)

laser = Jones_vector("laser").general_azimuth_ellipticity(
    azimuth=az_radian, ellipticity=el_radian
)

(a_pol, a_retarder) = (145 * degree_to_rad, 80 * degree_to_rad)

polarizer = Jones_matrix("polarizer").diattenuator_perfect(azimuth=a_pol)
retarder = Jones_matrix("retarder").retarder_linear(azimuth=a_retarder)
PSA: Jones_matrix = polarizer * retarder

output = PSA * laser

intensity = output.parameters.intensity(draw=False)

min = intensity.argmin()
max = intensity.argmax()
plt.pcolor(az_radian / degree_to_rad, el_radian / degree_to_rad, intensity)

# Verifying
max_i = intensity.flatten()[max]
min_i = intensity.flatten()[min]
# Plot max/min points
az_degree = az_radian.flatten() / degree_to_rad
el_degree = el_radian.flatten() / degree_to_rad
plt.scatter(az_degree[min], el_degree[min])
plt.scatter(az_degree[max], el_degree[max])
plt.show()